In [1]:
#5

In [2]:
import tensorflow as tf
import os
import time

/home/serverteam_1/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
%run "./Constants"

ERROR:root:File `'./Constants.py'` not found.


In [5]:
%run "./Data Ingest"

ERROR:root:File `'./Data Ingest.py'` not found.


In [6]:
%run "./Constructing the Model Graph"

ERROR:root:File `'./Constructing the Model Graph.py'` not found.


In [7]:
def restore_model(sess, checkpoint_dir, retry_sec=1):
  while True:
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
      tf.train.Saver().restore(sess, ckpt.model_checkpoint_path)
      global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
      return global_step
    print("No checkpoint file found in directory %s, retrying after %s sec..."%(checkpoint_dir, retry_sec))
    time.sleep(retry_sec)

In [8]:
from SyncUtils import syncDirs
import math

ModuleNotFoundError: No module named 'SyncUtils'

In [9]:
def model_eval(num_epochs, wait_between_epochs):
  """ Every wait_between_epochs seconds, loads our model from checkpoint and computes its accuracy on the full validation set. """
  # Download validation data from S3; our dataset is in a public bucket, so we don't
  # need to pass credentials when reading data
  download_files(worker_num=0, num_workers=1, bucket_name=BUCKET_NAME, prefix=EVAL_PREFIX,
                            region_name=S3_REGION, local_dir=LOCAL_DATA_DIR, use_authentication=False)  
  # Get a tuple (images, labels) corresponding to a batch of validation data
  tf.reset_default_graph()
  images, labels = get_batch(data_dir=LOCAL_DATA_DIR, batch_size=BATCH_SIZE, num_readers=1, num_preprocess_threads=2, file_prefix='validation')
  # Get model loss, accuracy on validation data
  _, accuracy = get_loss_and_accuracy(images, labels)  
  batches_per_epoch = int(math.ceil(NUM_VALIDATION_EXAMPLES / BATCH_SIZE))
  with tf.Session() as sess:  
    # Create a SummaryWriter that writes evaluation event files to the same directory as training event files
    summary_writer = tf.summary.FileWriter(LOCAL_EVENT_FILE_DIR, graph=sess.graph)
    # Sync event files back to DBFS for persistent storage
    syncDirs(src_dir=LOCAL_EVENT_FILE_DIR, dst_dir=DBFS_EVENT_FILE_DIR, sync_wait_secs=20)
    for epoch in xrange(num_epochs):
        try:
          # Restore model & get the number of training batches processed by the model as global_step
          global_step = restore_model(sess, checkpoint_dir=CHECKPOINT_DIR)
          # Compute model accuracy on the validation set
          accuracy_sum = 0.0
          for i in range(batches_per_epoch):  
            accuracy_sum += sess.run(accuracy)
          eval_accuracy = accuracy_sum / batches_per_epoch
          tf.logging.info("Global step %s, evaluation accuracy: %s"%(global_step, eval_accuracy))          
          # Write model accuracy to event file to be consumed by TensorBoard
          summary = tf.Summary()
          summary.value.add(tag='Evaluation accuracy', simple_value=eval_accuracy)
          summary_writer.add_summary(summary, global_step)
          # Wait before running next evaluation step
          time.sleep(wait_between_epochs)          
        except tf.errors.InvalidArgumentError as e:
          tf.logging.info("Caught exception %s"%e)
          tf.logging.info("Failed to restore checkpoint, retrying...")
          pass

In [10]:
model_eval(num_epochs=20, wait_between_epochs=30)


NameError: name 'download_files' is not defined